Преобразование данных модели, полученной интегрированием из центра, в сетку с равномерным шагом по массе. 

In [29]:
import math
import numpy as np
import pandas as pd

%matplotlib inline

import matplotlib as mpl
import matplotlib.pyplot as plt

In [30]:
import physics as ph
from Sun import SSM18 as s

In [100]:
def load_model(epoch):
    filename = 'FromCenter/1epoch%.02d.csv' % epoch
    return pd.read_csv(filename)

In [101]:
# load original model
epoch = 0
df = load_model(epoch)
df

,Unnamed: 0,Mass,Radius,Temperature,Density,Luminosity,Hydrogen,Opacity,Convection,Pressure,Velocity
0,0,0.000000e+00,0.000000e+00,1.333000e+07,94.554471,0.000000e+00,0.732,1.318613,0.0,1.730205e+17,8.561869e-11
1,1,1.335360e+26,6.960000e+07,1.333000e+07,94.554471,1.634396e+27,0.732,1.318613,1.0,1.730205e+17,8.561869e-11
2,2,1.735967e+26,7.656000e+07,1.332997e+07,94.554040,2.124711e+27,0.732,1.318620,0.0,1.730193e+17,8.561741e-11
3,3,2.220700e+26,8.352000e+07,1.332993e+07,94.553578,2.717979e+27,0.732,1.318627,0.0,1.730180e+17,8.561603e-11
4,4,2.797568e+26,9.048000e+07,1.332990e+07,94.553080,3.424001e+27,0.732,1.318634,0.0,1.730166e+17,8.561455e-11
...,...,...,...,...,...,...,...,...,...,...,...
13995,13995,1.989737e+33,6.262796e+10,5.465099e+03,0.000031,2.461129e+33,0.732,0.003113,0.0,2.349260e+07,4.899800e-49
13996,13996,1.989737e+33,6.262800e+10,5.460450e+03,0.000031,2.461129e+33,0.732,0.003109,0.0,2.344258e+07,4.849587e-49
13997,13997,1.989737e+33,6.262805e+10,5.455801e+03,0.000031,2.461129e+33,0.732,0.003105,0.0,2.339263e+07,4.799847e-49
13998,13998,1.989737e+33,6.262810e+10,5.451153e+03,0.000031,2.461129e+33,0.732,0.003101,0.0,2.334274e+07,4.750576e-49


In [130]:
x = df['Hydrogen'].values
r = df['Radius'].values
m = df['Mass'].values
t = df['Temperature'].values
d = df['Density'].values
l = df['Luminosity'].values
p = df['Pressure'].values
k = df['Opacity'].values
c = df['Convection'].values
v = df['Velocity'].values

In [131]:
# Число точек в исходной R-модели
dimR = len(df.index)
# Число точек в новой M-модели
dimM = 1001

In [132]:
# Шаг по массе
dm = m[-1]/(dimM-1) 

In [133]:
dm

1.989737388507742e+30

In [134]:
# Массивы для данных новой модели
m1 = np.zeros(dimM)
r1 = np.zeros(dimM)
t1 = np.zeros(dimM)
d1 = np.zeros(dimM)
l1 = np.zeros(dimM)
x1 = np.zeros(dimM)
k1 = np.zeros(dimM)
p1 = np.zeros(dimM)
c1 = np.zeros(dimM)
v1 = np.zeros(dimM)

In [135]:
m1[0] = m[0]
r1[0] = r[0]
l1[0] = l[0]
t1[0] = t[0]
d1[0] = d[0]
x1[0] = x[0]
k1[0] = k[0]
p1[0] = p[0]
c1[0] = c[0]
v1[0] = v[0]


In [136]:
j = 1 # index in dimM
i0 = 0
m1[j] = dm
for i in range(1, dimR):
    if m[i] <  m1[j]:
        continue
    f = (m1[j] - m[i-1])/(m[i] - m[i-1])
#     print(i, j, f, i - i0)
    i0 = i
    r1[j] = r[i-1]*(1 - f) + r[i]*f
    t1[j] = t[i-1]*(1 - f) + t[i]*f
    d1[j] = d[i-1]*(1 - f) + d[i]*f
    l1[j] = l[i-1]*(1 - f) + l[i]*f
    x1[j] = x[i-1]*(1 - f) + x[i]*f
    k1[j] = k[i-1]*(1 - f) + k[i]*f
    p1[j] = p[i-1]*(1 - f) + p[i]*f
    c1[j] = c[i-1]*(1 - f) + c[i]*f
    v1[j] = v[i-1]*(1 - f) + v[i]*f
    j += 1
    if j < dimM:
        m1[j] = j*dm
    

In [137]:
df1 = pd.DataFrame({'Mass': m1, 'Radius': r1, 'Temperature': t1, 'Density': d1, 
          'Luminosity': l1, 'Hydrogen': x1, 'Opacity': k1, 'Convection': c1, 'Pressure': p1, 'Velocity': v1})


In [138]:
df1

,Mass,Radius,Temperature,Density,Luminosity,Hydrogen,Opacity,Convection,Pressure,Velocity
0,0.000000e+00,0.000000e+00,1.333000e+07,94.554471,0.000000e+00,0.732,1.318613,0.0,1.730205e+17,8.561869e-11
1,1.989737e+30,1.720827e+09,1.323295e+07,93.267398,2.371017e+31,0.732,1.337635,0.0,1.694228e+17,8.186134e-11
2,3.979475e+30,2.170742e+09,1.317560e+07,92.506463,4.667612e+31,0.732,1.348390,0.0,1.673123e+17,7.971258e-11
3,5.969212e+30,2.487796e+09,1.312752e+07,91.867973,6.909218e+31,0.732,1.357124,0.0,1.655511e+17,7.795029e-11
4,7.958950e+30,2.741213e+09,1.308464e+07,91.297969,9.104061e+31,0.732,1.364692,0.0,1.639865e+17,7.640787e-11
...,...,...,...,...,...,...,...,...,...,...
996,1.981778e+33,5.478443e+10,8.884676e+05,0.065450,2.461129e+33,0.732,6.506261,1.0,7.982444e+12,2.801593e-22
997,1.983768e+33,5.565851e+10,7.779882e+05,0.053620,2.461129e+33,0.732,5.330260,1.0,5.726433e+12,5.617357e-23
998,1.985758e+33,5.672587e+10,6.475668e+05,0.040707,2.461129e+33,0.732,4.046600,1.0,3.618575e+12,6.097868e-24
999,1.987748e+33,5.818398e+10,4.769614e+05,0.025719,2.461129e+33,0.732,2.556699,1.0,1.683936e+12,1.506867e-25


In [139]:
_ = df1.to_csv(line_terminator='\n')
filename = "Mass/1epoch%.02d.csv" % epoch
print(filename)
csv_file = open(filename, "wt")
n = csv_file.write(_)
csv_file.close()

Mass/1epoch00.csv
